In [25]:
import coneClustering
import numpy as np
from sklearn.datasets import fetch_olivetti_faces, fetch_openml  # Similar, or use your ORL data
from sklearn.decomposition import PCA

## Olivetti face dataset

In [14]:
# Example: load dataset (fetch_olivetti_faces is a nice substitute for ORL if you don't have it locally)
faces = fetch_olivetti_faces(shuffle=True, random_state=42)
X = faces.data.T  # (d, n) --> exactly what your algorithm expects
true_labels = faces.target  # (n,)

mask = faces.target < 6
X_subset = faces.data[mask].T  # Shape (4096, 100)
true_labels_subset = faces.target[mask]


In [21]:
_, ssc_accuracy = coneClustering.baseline_ssc(X_subset, true_labels_subset, alpha=1e-3)
print(f"SSC Accuracy: {ssc_accuracy:.4f}")

length of cluster_labels:  60
SSC Accuracy: 0.4939


In [23]:
accuracy, reconstruction_error, neg_prop = coneClustering.iter_reg_coneclus(
    X_subset,
    K=6,   # 6 subjects
    r=5,    # start with r=5 or r=7
    true_labels=true_labels_subset,
    max_iter=100,
    random_state=42,
    alpha=2.3,
    ord=2
)

## RICC

In [24]:
accuracy

0.041146077970878346

## MNIST

In [ ]:
mnist = fetch_openml('mnist_784', version=1)
X_full = mnist.data.to_numpy()  # shape (70000, 784)
y_full = mnist.target.to_numpy().astype(int)  # shape (70000,)

# 2. Subset 500 images per digit
X_list = []
labels = []

for digit in range(5):
    idx = np.where(y_full == digit)[0]
    selected_idx = np.random.choice(idx, 50, replace=False)  # randomly select 500 images
    X_list.append(X_full[selected_idx])
    labels.append(np.full(len(selected_idx), digit))

X_subset = np.vstack(X_list)  # shape (5000, 784)
true_labels = np.concatenate(labels)  # shape (5000,)

# 3. Transpose for your algorithm (your method expects (d, n))
X_subset = X_subset.T  # Now (784, 5000)

print(f"X shape: {X_subset.shape}")
print(f"true_labels shape: {true_labels.shape}")


/Users/xrong8/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


X shape: (784, 35735)
true_labels shape: (35735,)


In [ ]:
# 4. Run your algorithm
accuracy, reconstruction_error, neg_prop = coneClustering.iter_reg_coneclus(
    X_subset,
    K=10,  # 10 digits
    r=5,   # Start small, you can tune this
    true_labels=true_labels,
    max_iter=100,
    random_state=42,
    alpha=0.1,
    ord=2
)

print(f"Accuracy (ARI): {accuracy:.4f}")
print(f"Reconstruction Error: {reconstruction_error:.4f}")
print(f"Proportion of negatives: {neg_prop:.4f}")

In [28]:
_, ssc_accuracy = coneClustering.baseline_ssc(X_subset, true_labels, alpha=1e-3)
print(f"SSC Accuracy: {ssc_accuracy:.4f}")

/Users/xrong8/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.128e+04, tolerance: 3.245e+02
  model = cd_fast.enet_coordinate_descent(
/Users/xrong8/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.091e+02, tolerance: 2.586e+02
  model = cd_fast.enet_coordinate_descent(
/Users/xrong8/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

length of cluster_labels:  5000
SSC Accuracy: 0.2337
